In [48]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

google_notes_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv'
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

notes = pd.read_csv(google_notes_csv).fillna('')
notes = notes[['Category','Categorization','Word','Definition']].copy()

definitions = pd.read_csv(google_definition_csv).fillna('')

ml_def = definitions.copy()
ml_notes = notes[notes['Category'].isin(['Machine Learning','Visualizations'])]

#from df_processing import notes_df_to_outline_html,final_dataset_for_markdown
#d_learning_notes = final_dataset_for_markdown(notes,definitions)
#d_learning_notes

In [ ]:
# DQ Rules
# No Duplication in definition.Word

In [50]:
rank_df = notes.drop_duplicates(['Category','Categorization']).reset_index(drop=True).reset_index().rename(columns={'index':'OG_RANK'})
rank_df

,OG_RANK,Category,Categorization,Word,Definition
0,0,Machine Learning,Problem Definition,Goal,Need this to Pull from Somewhere Else.
1,1,Machine Learning,Data Collection,Goal,Need this to Pull from Somewhere Else.
2,2,Machine Learning,Data Preperation,Goal,
3,3,Machine Learning,Feature Engineering,Goal,
4,4,Machine Learning,Feature Selection,Goal,
5,5,Machine Learning,Model,Goal,
6,6,Machine Learning,Training,Goal,
7,7,Machine Learning,Hyperparameter Tuning,Goal,
8,8,Machine Learning,Validation,Goal,
9,9,Machine Learning,Evaluation,Goal,


In [100]:
def final_dataset_for_markdown_v2(notes=None,
                               definitions=None,
                               export_location='/Users/derekdewald/Documents/Python/Github_Repo/Streamlit/DataDictionary/'):
    
    '''
    
    Function which helps to combined Notes and Definitions into a Single Combined Representation which can ultimately be used as a Learning Reference Tools.

    How this function Works:
    It takes the two sheets and attemps to Consolidate them together to make Final Dataset, generated as d_learning_notes.csv.

    Step 1: Find Definition for Category.
            notes.Category = definitions.Word
    Step 2: Find All Records for the Category are identical and Categorization is Definitions from Definition Sheet.
            notes.Category = defintions.Word and definitions.categorization='Definition'
    Step 3: Find All Records which

    
    #################





    date_created:20-Dec-25
    date_last_modified: 21-Dec-25
    classification:TBD
    sub_classification:TBD
    usage:
        final_dataset_for_markdown()


    ##############

    Has been tested for a Single Value - Machine Learning. Need to Validate once extending.

    ##############

    
    '''

    from data_d_dicts import links

    try:
        len(notes)
    except:
        notes = pd.read_csv(links['google_notes_csv'])

    try:
        len(definitions)
    except:
        definitions = pd.read_csv(links['google_definition_csv'])
    
    temp_def = definitions[['Category','Categorization','Word','Definition']].copy()

    # Step 1
    # Create Unique Category DF
    unique_cat_df = notes.drop_duplicates(['Category'])[['Category']]

    # Process temp_def such that 
    step1_df = unique_cat_df.merge(temp_def.drop('Category',axis=1),left_on='Category',right_on='Word',how='left')
    step1_df['Categorization'] = 'Definition'
    step1_df['Word'] = step1_df['Category'].copy()
    step1_df['Definition'] = step1_df['Definition'].fillna('Not Defined')


    # Step 2
    unique_cat_df['Categorization'] = 'Definition'
    step2_df = unique_cat_df.merge(temp_def,on=['Category','Categorization'],how='inner')

    

    final_df = pd.concat([step1_df,step2_df]).drop_duplicates()

    # Step 3
    unique_cat_cat = 'd'

    # Step 4
    mod_def = definitions[['Category','Categorization','Word','Definition']].copy()
    mod_def['Definition'] = mod_def['Word'] + ": " + mod_def['Definition']
    mod_def = mod_def.drop('Word',axis=1).reset_index(drop=True)
    mod_def = mod_def.rename(columns={'Categorization':'Word','Category':'Categorization'})
    
    # Merge Notes.Categorization and Notes.Word to Definitions.Category and Definitions.Categorization
    step_4df = notes.drop_duplicates(['Categorization','Word'])[['Category','Categorization','Word']].merge(mod_def,on=['Categorization','Word'],how='inner')

    # Can DO ORDER AT THE END. USING THE ORIGINAL DATASET and Category MERGE IN

    final_df = pd.concat([final_df,step_4df])

    rank_df1 = notes.drop_duplicates('Category')[['Category']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank1"})
    rank_df2 = notes.drop_duplicates(['Category','Categorization'])[['Category','Categorization']].reset_index(drop=True).reset_index().rename(columns={'index':"Rank2"})
    rank_df3 = notes.drop_duplicates(['Category','Categorization','Word']).reset_index(drop=True).reset_index().drop('Definition',axis=1).rename(columns={'index':'Rank3'})
    final_df = final_df.merge(rank_df1,on=['Category'],how='left').merge(rank_df2,on=['Category','Categorization'],how='left').merge(rank_df3,on=['Category','Categorization','Word'],how='left').fillna(0).sort_values(['Rank1','Rank2','Rank3']).reset_index(drop=True)

    return final_df

d = final_dataset_for_markdown_v2(notes,definitions)  
d

#d_learning_notes = final_dataset_for_markdown(notes,definitions)


,Category,Categorization,Word,Definition,Rank1,Rank2,Rank3
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...,0,0.0,0.0
1,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,0,0.0,0.0
2,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...,0,0.0,0.0
3,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...,0,0.0,0.0
4,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...,0,0.0,0.0
...,...,...,...,...,...,...,...
239,Mathematics,Definition,Mathematics,Not Defined,1,0.0,0.0
240,Mathematics,Definition,Algorithm,An algorithm is a procedure that unfolds over ...,1,0.0,0.0
241,Mathematics,Definition,Constraint,A constraint is something that restricts or pe...,1,0.0,0.0
242,Mathematics,Definition,Function,A function is a direct mathematical mapping: g...,1,0.0,0.0


In [80]:
notes

,Category,Categorization,Word,Definition
0,Machine Learning,Problem Definition,Goal,Need this to Pull from Somewhere Else.
1,Machine Learning,Problem Definition,Approach,
2,Machine Learning,Problem Definition,Important to Remeber,
3,Machine Learning,Problem Definition,Lesson Learnt,
4,Machine Learning,Problem Definition,Algorithm,
...,...,...,...,...
111,Machine Learning,Monitoring,Algorithm,
112,Machine Learning,Monitoring,Function,
113,Machine Learning,Monitoring,Procedure,
114,Machine Learning,Monitoring,TBD,


In [79]:
rank1 = d[['Category']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'RANK1'})
d[['Category','Categorization']].drop_duplicates(['Category','Categorization']).reset_index(drop=True).reset_index().rename(columns={'index':'RANK2'})

,RANK2,Category,Categorization
0,0,Machine Learning,Definition
1,1,Mathematics,Definition
2,2,Machine Learning,Model
3,3,Machine Learning,Validation


In [92]:
rank_df1

,Rank1,Category
0,0,Machine Learning
1,1,Mathematics


,Category,Categorization,Word,Definition,Rank1,Rank2,Rank3
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...,0,0.0,0.0
1,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,0,0.0,0.0
2,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...,0,0.0,0.0
3,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...,0,0.0,0.0
4,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...,0,0.0,0.0
...,...,...,...,...,...,...,...
239,Mathematics,Definition,Mathematics,Not Defined,1,0.0,0.0
240,Mathematics,Definition,Algorithm,An algorithm is a procedure that unfolds over ...,1,0.0,0.0
241,Mathematics,Definition,Constraint,A constraint is something that restricts or pe...,1,0.0,0.0
242,Mathematics,Definition,Function,A function is a direct mathematical mapping: g...,1,0.0,0.0


In [65]:
notes

,Category,Categorization,Word,Definition
0,Machine Learning,Problem Definition,Goal,Need this to Pull from Somewhere Else.
1,Machine Learning,Problem Definition,Approach,
2,Machine Learning,Problem Definition,Important to Remeber,
3,Machine Learning,Problem Definition,Lesson Learnt,
4,Machine Learning,Problem Definition,Algorithm,
...,...,...,...,...
111,Machine Learning,Monitoring,Algorithm,
112,Machine Learning,Monitoring,Function,
113,Machine Learning,Monitoring,Procedure,
114,Machine Learning,Monitoring,TBD,


In [3]:
definitions.head(1)

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class
0,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,,,,,,,


In [8]:
notes[notes['Categorization']=='Model']

,Category,Categorization,Word,Definition
42,Machine Learning,Model,Goal,
43,Machine Learning,Model,Approach,
44,Machine Learning,Model,Regularization,Automatically Included because it has a Record...
45,Machine Learning,Model,Important to Remeber,
46,Machine Learning,Model,Lesson Learnt,
47,Machine Learning,Model,Algorithm,
48,Machine Learning,Model,Function,
49,Machine Learning,Model,Procedure,
50,Machine Learning,Model,Constraint,


,Categorization,Word,Definition
0,Machine Learning,Definition,Area Under the Curve: AUC (Area Under the Curv...
1,Machine Learning,Definition,Autoregressive: Autoregressive models generate...
2,Machine Learning,Definition,Bias: Bias refers to the systematic error in a...
3,Machine Learning,Definition,Bias - Variance Trade Off: The bias-variance t...
4,Machine Learning,Definition,Cross Validation: Model evaluation technique u...
...,...,...,...
262,Best Linear Unbiased Estimator,Requirements,Linearity: Relationship between Independent an...
263,Best Linear Unbiased Estimator,Requirements,Independence.: Observations should be independ...
264,Best Linear Unbiased Estimator,Requirements,Homoscedasticity: Variance of the residuals sh...
265,Best Linear Unbiased Estimator,Requirements,Normality of Residuals: Residuals should be no...


,Category,Categorization,Word,Definition
0,Machine Learning,Model,Algorithm,Ada: Boosting technique that combines multiple...
1,Machine Learning,Model,Algorithm,AdaBoostClassifier:
2,Machine Learning,Model,Algorithm,AdaBoostRegressor:
3,Machine Learning,Model,Algorithm,AffinityPropagation:
4,Machine Learning,Model,Algorithm,"Agentic AI: Perform a Specific Task, Autonomou..."
...,...,...,...,...
220,Machine Learning,Model,Procedure,TransformedTargetRegressor:
221,Machine Learning,Validation,Function,Accuracy: Number of correctly classified examp...
222,Machine Learning,Validation,Procedure,Confusion Matrix: Table used to evaluate the p...
223,Machine Learning,Validation,Procedure,K Fold Cross Validation: Application of Cross ...


In [14]:
params = definitions[['Category','Categorization','Word','Definition']].rename(columns={
    
    
    'Word':'Notes','Categorization':'Word','Category':'Categorization'})
#
#
#params


,Categorization,Word,Notes,Definition
0,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...
1,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...
2,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...
3,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...
4,Machine Learning,Definition,Cross Validation,Model evaluation technique used to assess how ...
...,...,...,...,...
262,Best Linear Unbiased Estimator,Requirements,Linearity,Relationship between Independent and Dependent...
263,Best Linear Unbiased Estimator,Requirements,Independence.,Observations should be independent of each oth...
264,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.
265,Best Linear Unbiased Estimator,Requirements,Normality of Residuals,Residuals should be normally distributed.


In [77]:
definitions[definitions['Category']=='Model'][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
18,Model,Algorithm,Ada,Boosting technique that combines multiple weak...
19,Model,Algorithm,AdaBoostClassifier,
20,Model,Algorithm,AdaBoostRegressor,
21,Model,Algorithm,AffinityPropagation,
22,Model,Algorithm,Agentic AI,"Perform a Specific Task, Autonomously while le..."
...,...,...,...,...
235,Model,Procedure,StandardScaler,
236,Model,Procedure,TargetEncoder,
237,Model,Procedure,TfidfTransformer,
238,Model,Procedure,TransformedTargetRegressor,


In [76]:
notes.drop_duplicates(['Category','Categorization'])[['Category','Categorization']].merge(params

,Category,Categorization
0,Machine Learning,Problem Definition
8,Machine Learning,Data Collection
16,Machine Learning,Data Preperation
24,Machine Learning,Feature Engineering
32,Machine Learning,Feature Selection
42,Machine Learning,Model
51,Machine Learning,Training
59,Machine Learning,Hyperparameter Tuning
67,Machine Learning,Validation
75,Machine Learning,Evaluation


In [53]:
notes[notes['Categorization']=='Model']

,Category,Categorization,Word,Notes
42,Machine Learning,Model,Goal,
43,Machine Learning,Model,Approach,
44,Machine Learning,Model,Regularization,Automatically Included because it has a Record...
45,Machine Learning,Model,Important to Remeber,
46,Machine Learning,Model,Lesson Learnt,
47,Machine Learning,Model,Algorithm,
48,Machine Learning,Model,Function,
49,Machine Learning,Model,Procedure,
50,Machine Learning,Model,Constraint,


In [51]:
definitions

,Category,Categorization,Word,Definition,Notes,Link,Image,Markdown Equation,Dataset Size,Learning Type,Algorithm Class
0,Machine Learning,Definition,Area Under the Curve,AUC (Area Under the Curve) measures a model’s ...,,,,,,,
1,Machine Learning,Definition,Autoregressive,Autoregressive models generate outputs one ste...,,,,,,,
2,Machine Learning,Definition,Bias,Bias refers to the systematic error in a model...,,,https://raw.githubusercontent.com/derek-dewald...,### **Unbiased Estimator Equation**\nA paramet...,,,
3,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...,,,https://raw.githubusercontent.com/derek-dewald...,,,,
4,Machine Learning,Definition,Cross Validation,Model evaluation technique used to assess how ...,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
262,Best Linear Unbiased Estimator,Requirements,Linearity,Relationship between Independent and Dependent...,,,,,,,
263,Best Linear Unbiased Estimator,Requirements,Independence.,Observations should be independent of each oth...,,,,,,,
264,Best Linear Unbiased Estimator,Requirements,Homoscedasticity,Variance of the residuals should be constant.,,,,,,,
265,Best Linear Unbiased Estimator,Requirements,Normality of Residuals,Residuals should be normally distributed.,,,,,,,


In [50]:
notes.drop_duplicates(['Category','Categorization']).drop(['Word','Notes'],axis=1).

,Category,Categorization
0,Machine Learning,Problem Definition
8,Machine Learning,Data Collection
16,Machine Learning,Data Preperation
24,Machine Learning,Feature Engineering
32,Machine Learning,Feature Selection
42,Machine Learning,Model
51,Machine Learning,Training
59,Machine Learning,Hyperparameter Tuning
67,Machine Learning,Validation
75,Machine Learning,Evaluation


In [134]:
def search_df_word(df,word,columns=None,case_sensitive=False,print_=True):
    '''
    
    
    '''

    # If User does not define which columns they wish to search, search all.
    if not columns:
        columns = df.columns.tolist()

    final_dict = {}
    final_df = pd.DataFrame()

    for column in columns:
        match_df = df[df[column].fillna('').str.contains(word,case=case_sensitive)]
        if len(match_df)==0:
            final_dict[column] = 'No Matches'
        else:
            final_word = ""
            for word in match_df[column]:
                final_word += word + ","
        
            final_dict[column] = final_word
            final_df = pd.concat([final_df,match_df])

    if print_:
        try:
            print(final_df)
        except:
            pass
            
    return final_df, final_dict

#final_df,final_dict = search_df_word(d_learning_notes,'regularization')
final_df,final_dict = search_df_word(d_learning_notes,'lasso')

    

### Examples of How Information is Included

#### Example Step 1A: Machine Learning

In [17]:
def step1a_example(df=d_learning_notes,
                   notes=notes,
                   definitions=definitions):
    
    print(f'Notes DataFrame:\nIncludes All Items with a record in Category')
    print(f'As Illustrated below by a Value Count\n')
    print(df['Category'].value_counts())

    values = df.drop_duplicates("Category")["Category"].tolist()

    print('Definition DataFrame:\nLooks for Records with the Category

    

    print(f'\nFinal D Learning Notes')
    display(d_learning_notes[d_learning_notes['Word'].isin(values)])

step1a_example()

Notes DataFrame:
Includes All Items with a record in Category
As Illustrated below by a Value Count

Category
Machine Learning    373
Mathematics           7
Name: count, dtype: int64

Final D Learning Notes


,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
373,Mathematics,Definition,Mathematics,Not Defined


In [ ]:
def step1b_example(word,
                   df=d_learning_notes,
                   notes=notes,
                   definitions=definitions):
    


STEP1A_EXAMPLE()

In [149]:
# Note there is No Explicit Record in this Sheet, simply the fact that it has a Value in Category
notes[notes['Category']=='Machine Learning']['Category'].value_counts()

Category
Machine Learning    115
Name: count, dtype: int64

In [130]:
# Record in DEFINITIONS Sheet
definitions[definitions['Word']=='Machine Learning'][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
7,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...


In [148]:
# Record in NOTES Sheet
d_learning_notes[d_learning_notes['Word']=='Machine Learning']

,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...


#### Example Step 1B: Bias - Variance Trade Off

In [146]:
definitions[definitions['Word'].str.contains('bias -',case=False)][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
3,Machine Learning,Definition,Bias - Variance Trade Off,The bias-variance trade-off is a key concept i...


##### Example 2: Linear Regression

In [24]:
# Linear Regression is Stored as a Model, Algorithm
definitions[definitions['Word']=='Linear Regression'][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
109,Model,Algorithm,Linear Regression,Models the relationship between an independent...


In [26]:
# It is included because there is a Corresponding Record in Notes for Model and Algorithm
notes[(notes['Categorization']=='Model')&(notes['Word']=='Algorithm')]

,Category,Categorization,Word,Notes
47,Machine Learning,Model,Algorithm,


In [20]:
# It appears as a record in Notes, Along with All other Models
d_learning_notes[d_learning_notes['Notes'].fillna("").str.contains('Linear Regression',case=False)]

,Category,Categorization,Word,Notes
164,Machine Learning,Model,Algorithm,Linear Regression: Models the relationship bet...


##### Example 2: Activation Function

In [12]:
# It is stored as a Part of Training, and a Definition, which is something we need to understand Training.
definitions[definitions['Category']=='Training'][['Category','Categorization','Word','Definition']]

,Category,Categorization,Word,Definition
254,Training,Definition,Activation Function,A mathematical function applied to a neuron’s ...
255,Training,Definition,Loss Function,A loss function is a mathematical function tha...
256,Training,Definition,Optimizer,Optimization algorithm that updates a model’s ...


In [29]:
# It is included because there is a Corresponding Categorization Training. 
# Note this is a Unique Example, because it is the Special Type DEFINITION, which are automatically Included and DO NOT need to be included
notes[(notes['Categorization']=='Training')]

,Category,Categorization,Word,Notes
51,Machine Learning,Training,Goal,
52,Machine Learning,Training,Approach,
53,Machine Learning,Training,Important to Remember,
54,Machine Learning,Training,Lesson Learnt,
55,Machine Learning,Training,Algorithm,
56,Machine Learning,Training,Function,
57,Machine Learning,Training,Procedure,
58,Machine Learning,Training,TBD,


In [28]:
# It appears as a record in Notes, Along with All other Models
d_learning_notes[d_learning_notes['Notes'].fillna("").str.contains('Activation Function:',case=False)]

,Category,Categorization,Word,Notes
295,Machine Learning,Training,Definition,Activation Function: A mathematical function a...


['Problem Definition',
 'Data Collection',
 'Data Preperation',
 'Feature Engineering',
 'Feature Selection',
 'Model',
 'Training',
 'Hyperparameter Tuning',
 'Validation',
 'Evaluation',
 'Bias, Fairness and Ethics',
 'Model Summarization/ Model Interpretability',
 'Deployment',
 'Monitoring']